In [1]:
!pip install -U bitsandbytes

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import login

login(token="hf_.....................") 

model_name = "VyDat/llama3-8b-vietnamese-multi"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.float16
)

SYS_INSTRUCT = """
Bạn là một trợ lý AI thông minh, có khả năng trả lời các câu hỏi bằng tiếng Việt một cách thân thiện và tự nhiên. 
Nếu câu hỏi liên quan đến ngôn ngữ ký hiệu Việt Nam (VSL), hãy chuyển đổi hoặc giải thích ký hiệu sang ngôn ngữ nói một cách chính xác. 
Cung cấp câu trả lời ngắn gọn, hữu ích và phù hợp với ngữ cảnh của câu hỏi.
"""

def format_prompt(user_input):
    messages = [
        {"role": "system", "content": SYS_INSTRUCT},
        {"role": "user", "content": user_input}
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Vòng lặp inference
def main():
    while True:
        user_input = input()
        if user_input.lower() == 'quit':
            print('Tạm biệt\n')
            break

        prompt = format_prompt(user_input)

        inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to("cuda")

        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True,
            output_scores=False
        )

        # Cắt phần prompt để chỉ lấy phần sinh mới
        generated_ids = outputs.sequences[0][inputs["input_ids"].shape[-1]:]
        assistant_response = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

        print(assistant_response)
        print('\n\n')

if __name__ == "__main__":
    main()

2025-06-24 15:25:31.858084: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750778731.881247     162 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750778731.886551     162 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

 chào


Xin chào bạn!





 trong ngôn ngữ kí hiệu câu "tôi màu hồng thích" có nghĩa là gì trong ngôn ngữ nói


...

Tôi biết ngôn ngữ ký hiệu VSL (Việt Nam Thể hiện Ngôn ngữ Kí hiệu). VSL là hệ thống ký hiệu được sử dụng để giúp đỡ những người Việt Nam bị khó khăn khi nói, như những người bị khiếm thính, khiếm nói hoặc những người muốn sử dụng ngôn ngữ ký hiệu trong giao tiếp.

Ngôn ngữ ký hiệu VSL bao gồm các ký hiệu cơ bản như các phần thể hiện trên mặt, trên tay và trong miệng. Chúng được sử dụng để diễn tả các âm thanh, từ ngữ và ngữ cảnh trong tiếng Việt.

Nếu bạn có bất kỳ câu hỏi nào liên quan đến ngôn ngữ ký hiệu VSL, tôi sẵn sàng giúp đỡ bạn.





 how old are you có nghĩa là gì


"How old are you" trong tiếng Việt có nghĩa là "Bạn bao nhiêu tuổi".





 quit


Tạm biệt

